In [1]:
spark

## Read Data from HDFS using PySpark

In [15]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .getOrCreate()

# Path to the CSV file on HDFS
file_path = "hdfs:///user/student/combined_raw_data.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame Schema
df.printSchema()

root
 |-- Line#: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Water Content (m3/m3): double (nullable = true)
 |-- Solar Radiation (W/m2): double (nullable = true)
 |-- Rain (mm): double (nullable = true)
 |-- Temperature (Celcius): double (nullable = true)
 |-- RH (%): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Gust Speed (m/s): double (nullable = true)
 |-- Wind Direction (Degree): double (nullable = true)
 |-- Dew Point (Celcius): double (nullable = true)



## Check Null Values with PySpark DataFrame

In [16]:
from pyspark.sql.functions import col, count, when, isnan

null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0]

# Print null counts for each column
for column, null_count in zip(df.columns, null_counts):
    print(f"{column}: {null_count} null values")

Line#: 0 null values
Date: 0 null values
Time: 0 null values
Water Content (m3/m3): 0 null values
Solar Radiation (W/m2): 0 null values
Rain (mm): 0 null values
Temperature (Celcius): 0 null values
RH (%): 0 null values
Wind Speed (m/s): 0 null values
Gust Speed (m/s): 0 null values
Wind Direction (Degree): 0 null values
Dew Point (Celcius): 0 null values


## Count Datasets rows

In [17]:
df.count()

160418

## Drop Duplicate Datas

In [18]:
df = df.select("*").distinct()

## Count Datasets rows after dropping Duplicated Datas

In [19]:
# There isn't any duplicate datas.
df.count()

160418

## Convert Date Datatype from String to Date in (dd-MM-yyyy)

In [20]:
#import library
from pyspark.sql.functions import to_date, date_format

df = df.withColumn("Date", to_date("Date", "yy/MM/dd"))
df = df.withColumn("Date", date_format("Date", "yyyy-MM-dd"))

In [21]:
df.show(5)

# Print the schema of the selected columns
df.printSchema()

+-----+----------+--------+---------------------+----------------------+---------+---------------------+------+----------------+----------------+-----------------------+-------------------+
|Line#|      Date|    Time|Water Content (m3/m3)|Solar Radiation (W/m2)|Rain (mm)|Temperature (Celcius)|RH (%)|Wind Speed (m/s)|Gust Speed (m/s)|Wind Direction (Degree)|Dew Point (Celcius)|
+-----+----------+--------+---------------------+----------------------+---------+---------------------+------+----------------+----------------+-----------------------+-------------------+
| 2419|2021-05-09|09:30:00|               0.2837|                 326.0|      0.0|                31.61|  75.7|             0.0|             0.0|                  108.0|              26.83|
| 2454|2021-05-09|12:25:00|               0.2837|                 411.0|      0.0|                33.47|  66.4|             0.0|             0.0|                   80.0|              26.37|
| 2513|2021-05-09|17:20:00|               0.2834| 

In [27]:
# Path to save the single CSV file
output_file = "/home/student/Data/weather-datasets/phnom_penh/Big Data Weather Prediction Capstone Project/cleaned.csv"

# Save the DataFrame to a single CSV file
df.coalesce(1).write.option("header", True).mode("overwrite").csv("file:///tmp/intermediate_csv")

# Rename the single part file to `cleaned.csv`
import os
import shutil

# Intermediate directory where the file is saved
intermediate_dir = "/tmp/intermediate_csv/"

# Find the part file and rename it to `cleaned.csv`
for file_name in os.listdir(intermediate_dir):
    if file_name.startswith("part-"):
        shutil.move(
            os.path.join(intermediate_dir, file_name),
            output_file
        )
        break

# Clean up the intermediate directory
shutil.rmtree(intermediate_dir)

print(f"DataFrame saved as a single CSV file at {output_file}")

DataFrame saved as a single CSV file at /home/student/Data/weather-datasets/phnom_penh/Big Data Weather Prediction Capstone Project/cleaned.csv
